In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
from dotenv import load_dotenv
boto_session = Session()
load_dotenv()

region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "auditing_crew_getting_started_new_configured"
response = agentcore_runtime.configure(
    entrypoint="src/auditing_crew.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    region=region,
    agent_name=agent_name
)
response

In [ ]:
launch_result = agentcore_runtime.launch()

In [ ]:
!uv sync --active --force-reinstall

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

In [ ]:
import boto3
import json
from dotenv import load_dotenv
load_dotenv()

bedrock_agent_runtime = boto3.client(
    service_name='bedrock-agent-runtime',
    region_name='us-east-1'  # e.g., 'us-east-1', 'us-west-2'
)


def retrieve_and_generate(knowledge_base_id, query, model_id):
    response = bedrock_agent_runtime.retrieve_and_generate(
        input={
            'text': query
        },
        retrieveAndGenerateConfiguration={
            'type': "KNOWLEDGE_BASE",
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': knowledge_base_id,
                'modelArn': model_id,
            }
        }
    )

    return response


knowledge_base_id = 'O3IBGYPABF'
query = 'What information do you have about compliance guidelines on cash flow statement?'
# or another supported model
model_id = 'arn:aws:bedrock:us-east-1:992382746523:inference-profile/us.anthropic.claude-3-7-sonnet-20250219-v1:0'

try:
    result = retrieve_and_generate(knowledge_base_id, query, model_id)

    # Extract the generated answer
    answer = result['output']['text']
    print(f"Answer: {answer}")

    # Retrieve citation information
    citations = result.get('citations', [])
    for citation in citations:
        print(f"Citation: {citation}")

except Exception as e:
    print(f"Error: {str(e)}")

Answer: According to the information provided, an entity should present a statement of cash flows in accordance with Ind AS 7, Statement of Cash Flows. This is mentioned as part of the complete set of financial statements that an entity must prepare.

For cash flow statements, there's a specific example provided regarding foreign currency transactions. When an entity has bank balance in foreign currency, it should reconcile any unrealized exchange gains or losses. The example shows that for statement of cash flows purposes, unrealized exchange gains should be excluded from the cash flow calculation, and a reconciliation should be provided between the cash and cash equivalents as per the statement of cash flows and as per the balance sheet. The statement of cash flows is one of the required components in a complete set of financial statements, along with:
- A balance sheet as at the end of the period
- A statement of profit and loss for the period
- Statement of changes in equity for th

In [ ]:
from botocore.config import Config

# Configure retries and timeout
config = Config(
    retries={
        'max_attempts': 10,  # Increase max retries to 10 (default is 4)
        'mode': 'adaptive'   # Options: 'legacy', 'standard', 'adaptive'
    },
    connect_timeout=600,      # Connection timeout in seconds (default is 60)
    read_timeout=3000         # Read timeout in seconds (default is 60)
)

In [ ]:
import boto3
import json
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region,
    config=config
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "Artificial Intelligence in Healthcare"})
)

response_body = boto3_response['response'].read()
response_data = json.loads(response_body)
response_data

In [20]:
from textractor import Textractor
from textractor.data.constants import TextractFeatures

extractor = Textractor(profile_name="default")

document = extractor.start_document_analysis(
    file_source="s3://textract-console-ap-south-1-2901ebb3-1ec7-4e10-8813-6d3b48710e5/Integrated_Report_and_Annual_Financial_Statements.pdf",
    features=TextractFeatures.TABLES,
    s3_output_path="s3://textract-console-ap-south-1-2901ebb3-1ec7-4e10-8813-6d3b48710e5/output/",
)

In [18]:
from textractor.visualizers.entitylist import EntityList
from textractor.data.constants import TextractFeatures
from textractor.data.text_linearization_config import TextLinearizationConfig
from textractor.data.markdown_linearization_config import MarkdownLinearizationConfig

In [13]:
document

In [25]:
text = document.to_markdown(MarkdownLinearizationConfig(
    table_column_separator="|", table_row_separator="\n"))


with open("output.md", "w", encoding="utf-8", newline="\n") as f:
    f.write(text)

In [55]:
import requests
import base64
import json
import uuid

# Generate a random UUID (version 4)
unique_id = str(uuid.uuid4())

url = "https://pgrq5idqw4.execute-api.ap-south-1.amazonaws.com/test/upload"

with open(r"F:\udemy\langchain\langgraph\aws-hackathon-backend\src\knowledge\tcs_balance_sheet_fy25.pdf", "rb") as f:
    file_content_1 = base64.b64encode(f.read()).decode()
with open(r"F:\udemy\langchain\langgraph\aws-hackathon-backend\src\knowledge\TCS_cashflows_fy26.pdf", "rb") as f:
    file_content_2 = base64.b64encode(f.read()).decode()
with open(r"F:\udemy\langchain\langgraph\aws-hackathon-backend\src\knowledge\tcs_income_fy25.pdf", "rb") as f:
    file_content_3 = base64.b64encode(f.read()).decode()

payload = {
        "id": unique_id,
        "files":[
            {
                "file_content": file_content_1,
                "type": "balance_sheet",
            },
            {
                "file_content": file_content_2,
                "type": "cash_flows",
            },
            {
                "file_content": file_content_3,
                "type": "income_statement",
            },


        ]
    }


headers = {"Content-Type": "application/json"}

response = requests.post(url, headers=headers, data=json.dumps(payload))
print(response.json())

{'message': 'Service Unavailable'}
